In [133]:
user_msg = "I'm struggling with anxiety and depression. I live in Berlin-Mitte and prefer sessions in German. I'm looking for cognitive behavioral therapy or psychotherapy. My email is patient@example.com and my name is Alex Mueller. I'm publically insured in Germany."

In [134]:
import sys
import time
!{sys.executable} -m pip install openai python-dotenv requests reportlab


You should consider upgrading via the 'C:\Users\erikd\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [135]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [136]:
system_prompt_structured_request_transform = """You are an assistant that extracts structured search data for a doctor search API.
You only return valid JSON matching the following structure:

{
  "patient_name": string // patient name
  "patient_email": string // patient email
  "dist": int, // max distance in km + 1
  "lon": string, // longitude
  "lat": string, // latitude
  "fg": string, // filter by field code
  "zb": string, // filter by additional field code ("Ärztliche Zusatzbezeichungen")
  "bf": string, // filter by accessibility code
  "S": string, // filter by therapist preferred sex "F" for female, "M" for male
  "fs": string, // filter by language. each has a code. too many to list here, but English is "3"
  "pta": string, // filter by psychotherapy target group "E" for adults, "K" for children
  "pts": string, // filter by psychotherapy mode "E" for 1o1 therapy, "G" for group
  "ptv": string, // filter by psychotherapy foundation. "S" for systemic, "T" for depth psychology, "A" for analytical, "V" for behavioural
  "n": string, // filter by therapist name
}

Omit any filter fields you don't know or are not relevant.

Use default values when necessary. If city is mentioned, guess lat/lon.
"""

In [137]:
system_prompt_therapist_response_categorization = """You are tasked with a process involving email responses from therapists regarding the availability of psychotherapy spots. There may be parts of the patient's requests at the end. Ignore that.'

Categorize the relevant information as follows:

- 'no': If the therapist indicates they have no capacity or availability, either explicitly or implicitly. This also applies if they are temporarily blocked or they are on vacation.

- 'yes': If the therapist confirms any type of availability. Or mentions there is a free therapy spot. Or offers a first meeting. Or asks for more info to schedule a meeting such as telephone numbers or schedules.

- 'invalid_request': If the therapist either feels like they aren't the right person to talk to, doesn't do psychotherapy or the age group doesn't match. Or if there was an error delivering an email for example SMTP errors, invalid target emails, mailbox full, mailbox unavailable, etc.

- 'waiting_list': If the therapist currently doesn't have a spot but explicitly offers adding the patient to a waiting list. Offering alternatives does not fulfill this condition. And mentioning a full waiting list also does not fulfill this condition.

- 'automatic': If it is an automated response confirming recept or informing about processing times, contact guidelines, vacation times or other reasons.

- 'neither': If you cannot draw a conclusion about availability from the email.

Example:

Email: 'Dear Ms. Venti, Thank you for your inquiry. Unfortunately, I cannot offer you a free therapy or waitlist spot at this time. I suggest that you use the following websites during your search for a therapy spot: www.psych-info.de'

Relevant Information: 'Unfortunately, I cannot offer you a free therapy or waitlist spot at this time.'

Categorization: 'no'

Reason: The therapist explicitly states that they cannot offer a therapy or waitlist spot at this time.

Output in this format:
{"catgory":"<category>","availability_context":"<context>"}

availability_context is a short summary of the availability information. In case of a waiting list offer with a timeframe, add this timeframe.
"""

In [139]:
import json

def generate_structured_request(user_input):
    resp = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt_structured_request_transform},
            {"role": "user", "content": user_input}
        ],
        temperature=0.2,
    )
    return json.loads(resp.choices[0].message.content)


In [140]:
import json

def find_therapists(data):
    # simplified. in production, would use therapist search api integration such as https://github.com/Vector-Hector/kvberlin
    with open("therapist_api_example.json", "r", encoding="utf-8") as f:
        return json.load(f)

In [141]:
import os
import random

time_out_length = 7 # 7 days till time out

def email_therapist(therapist):
    # simplified. in production, would use callback+timeout
    if random.random() < 0.3:
        return {
            "error": "time_out"
        }

    response_dir = "example_responses"
    files = [f for f in os.listdir(response_dir) if f.endswith(".json")]
    if not files:
        raise FileNotFoundError("No JSON files found in example_responses/")

    chosen_file = random.choice(files)
    path = os.path.join(response_dir, chosen_file)

    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

In [142]:
def categorize_therapist_response(therapist_response):
    if "error" in therapist_response and therapist_response["error"] is not None:
        if therapist_response["error"] == "time_out":
            return {
                "category": "time_out",
                "availability_context": "Therapist did not respond within " + str(time_out_length) + " days."
            }

    resp = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt_therapist_response_categorization},
            {"role": "user", "content": therapist_response["body"]["plain"]}
        ],
        temperature=0.2,
    )
    return json.loads(resp.choices[0].message.content)



In [143]:
structured = generate_structured_request(user_msg)

In [144]:
therapists = find_therapists(structured)

In [145]:
import time
from datetime import datetime

therapy_found = False

categorized = []

for therapist in therapists:
    request_time = datetime.now().strftime("%d.%m.%Y %H:%M")
    response = email_therapist(therapist)

    cat = categorize_therapist_response(response)

    categorized.append({
        "request_time": request_time,
        "therapist": therapist,
        "response": response,
        "categorization": cat,
    })

    time.sleep(0.5)

In [148]:
if "yes" in [cat["categorization"]["category"] for cat in categorized]:
    print("A therapist has been found!")

In [179]:
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.platypus import Image
from collections import Counter

patient_name = structured["patient_name"]
patient_email = structured["patient_email"]

def generate_pdf(data, filename="output.pdf"):
    doc = SimpleDocTemplate(filename, pagesize=A4)
    styles = getSampleStyleSheet()
    styles.add(ParagraphStyle(name="LeftTitle", parent=styles["Title"], alignment=0))  # 0 = left
    elements = []

    # Main header

    img = Image("therapynowlogo.png")  # adjust size
    img.hAlign = "RIGHT"
    target_height = 30  # in points

    # Scale width proportionally
    aspect = img.imageWidth / img.imageHeight
    img.drawHeight = target_height
    img.drawWidth = target_height * aspect
    elements.append(img)  # insert at top

    elements.append(Spacer(1, 20))
    elements.append(Paragraph("Protocol: On the Search for an Outpatient Psychotherapy Slot with Statutory Health Insurance-Approved Psychotherapists", styles['LeftTitle']))
    elements.append(Spacer(1, 20))
    elements.append(Paragraph("Created using Therapy now. This protocol's purpose is to prove that the patient, " + patient_name + " actively searched for suitable therapists with available therapy spots. After the patient reached out to us, we searched for suitable therapists and contacted " + str(len(data)) + " of them. Here is a summary of the responses:", styles['Normal']))
    elements.append(Spacer(1, 20))

    counts = Counter(element["categorization"]["category"] for element in data)
    category_names = {
        "no": "stated they have no available therapy spots",
        "yes": "stated they do have available therapy spots",
        "waiting_list": "offered to add the patient to a waiting list",
        "time_out": "did not respond within a reasonable time frame",
        "automatic": "only responded with an auto-generated response such as holiday notices",
        "neither": "could not be categorized",
        "invalid_request": "stated they are not the right person to contact",
    }

    summary_data = [[counts[key], category_names[key]] for key in counts]

    summary_table = Table(summary_data, colWidths=[10, 440])
    summary_table.setStyle(TableStyle([
        ('BOX', (0,0), (-1,-1), 1, colors.transparent),
        ('GRID', (0,0), (-1,-1), 0.5, colors.transparent),
        ('FONT', (0,0), (-1,-1), 'Helvetica', 10)
    ]))
    elements.append(summary_table)
    elements.append(Spacer(1, 20))

    elements.append(Paragraph("Patient name: " + patient_name, styles['Normal']))
    elements.append(Spacer(1, 5))

    elements.append(Paragraph("Patient email: " + patient_email, styles['Normal']))
    elements.append(Spacer(1, 5))

    i = 0

    for element in data:
        category = element["categorization"]["category"]

        if category == "invalid_request":
            continue # not viable for reimbursement if we contacted the wrong person

        date_of_contact = element["request_time"]
        type_of_contact = "E-Mail"
        therapist_name = element["therapist"]["n"]
        therapist_email = element["therapist"]["e"]
        therapist_phone = element["therapist"]["te"]
        therapist_address = element["therapist"]["st"] + " " + element["therapist"]["ha"] + ", " + element["therapist"]["pl"] + " " + element["therapist"]["o"]

        availability_information = element["categorization"]["availability_context"]

        elements.append(Paragraph(f"Therapist #{i + 1}", styles['Heading2']))
        data = [
            ["Date and time of contact", date_of_contact],
            ["Type of contact", type_of_contact],
            ["Name of the psychotherapist", therapist_name],
            ["Address of the psychotherapist", therapist_address],
            ["E-Mail of the psychotherapist", therapist_email],
            ["Phone number of the psychotherapist", therapist_phone],
            ["Information on potential therapy spots", availability_information]
        ]
        data = [[Paragraph(a, styles["Normal"]), Paragraph(b, styles["Normal"])] for a, b in data]

        table = Table(data, colWidths=[150, 300])
        table.setStyle(TableStyle([
            ('BOX', (0,0), (-1,-1), 1, colors.black),
            ('GRID', (0,0), (-1,-1), 0.5, colors.grey),
            ('FONT', (0,0), (-1,-1), 'Helvetica', 10)
        ]))
        elements.append(table)
        elements.append(Spacer(1, 20))
        i+=1


    elements.append(Spacer(1, 20))
    elements.append(Paragraph("Patient signature: _______________________", styles['Normal']))
    elements.append(Spacer(1, 5))

    doc.build(elements)

    return summary_data

summary = generate_pdf(categorized, "therapists.pdf")



In [187]:
# simulate writing email to the patient
email = "Hi " + patient_name + """,

we have been reaching out to """ + str(len(categorized)) + " therapists and these are the responses: \n\n"

i = 0
for num, summary_name in summary:
    email += str(num) + " " + summary_name
    if i != len(summary) - 1:
         email += ", "
    else:
        email += ". \n"
    i+=1

email += """
We attached an important document to this email. You can use it to a request reimbursement procedure with your health insurer in case you have been looking for a therapy spot without success.

Our agent can guide you through the rest of the process.

Thank you for choosing us!
Your Therapy now team
"""

print(email)

Hi Alex Mueller,

we have been reaching out to 10 therapists and these are the responses: 

6 stated they have no available therapy spots, 3 did not respond within a reasonable time frame, 1 offered to add the patient to a waiting list. 

We attached an important document to this email. You can use it to a request reimbursement procedure with your health insurer in case you have been looking for a therapy spot without success.

Our agent can guide you through the rest of the process.

Thank you for choosing us!
Your Therapy now team

